In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Turn off TensorFlow warning messages in program output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
training_data_file = '../data/stratified_training_data/1992-2015_training_data_raw_n100000_ks_pval0.3.1.csv'
test_data_file = '../data/stratified_training_data/1992-2015_training_data_raw_n100000_ks_pval0.3.2.csv'

# Datatypes for dataframe loading
dtypes = {
    'weather_bin_time': object,
    'lat': float,
    'lon': float,
    'air.sfc': float,
    'rhum.2m': float,
    'dpt.2m': float,
    'pres.sfc': float,
    'uwnd.10m': float,
    'vwnd.10m': float,
    'veg': float,
    'dlwrf': float,
    'dswrf': float,
    'lcdc': float,
    'hcdc': float,
    'mcdc': float,
    'hpbl': float,
    'ulwrf.sfc': float,
    'ignition': float,
    'weather_bin_day': int,
    'weather_bin_month': int,
    'weather_bin_year': int
}

# Features to use during training 
features = [
    'weather_bin_month',
    'weather_bin_year',
    'air.sfc',
    'rhum.2m',
    'dpt.2m',
    'pres.sfc',
    'uwnd.10m', 
    'vwnd.10m',
    'veg',
    'lat',
    'lon',
]

In [3]:
training_data = pd.read_csv(training_data_file, dtype=dtypes)
testing_data = pd.read_csv(test_data_file, dtype=dtypes)

In [4]:
# count number of observations in each class
no_ignition_count = len(training_data[training_data["ignition"] == 0])
ignition_count = len(training_data) - no_ignition_count
print('Total training observations: {:.3E}'.format(len(training_data)))
print(' Ignitions count: {:.3E}'.format(ignition_count))
print(' Non ignitions count: {:.3E}'.format(no_ignition_count))

class_weight = no_ignition_count / ignition_count

print()

no_ignition_count = len(testing_data[testing_data["ignition"] == 0])
ignition_count = len(testing_data) - no_ignition_count
print('Total testing observations: {:.3E}'.format(len(testing_data)))
print(' Ignitions count: {:.3E}'.format(ignition_count))
print(' Non ignitions count: {:.3E}'.format(no_ignition_count))

Total training observations: 5.672E+04
 Ignitions count: 2.607E+03
 Non ignitions count: 5.411E+04

Total testing observations: 5.672E+04
 Ignitions count: 2.641E+03
 Non ignitions count: 5.408E+04


In [5]:
pd.options.mode.chained_assignment = None  # default='warn'

# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data[features].values

training_ignition = pd.DataFrame()
training_ignition[['no_ignition']] = abs(training_data[['ignition']] - 1)
training_ignition[['ignition']] = training_data[['ignition']]
Y_training = np.array(training_ignition)

# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = testing_data[features].values

testing_ignitions = pd.DataFrame()
testing_ignitions[['no_ignition']] = abs(testing_data[['ignition']] - 1)
testing_ignitions[['ignition']] = testing_data[['ignition']]
Y_testing = testing_ignitions.values

pd.options.mode.chained_assignment = 'warn'  # default='warn'

In [6]:
# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
#Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
#Y_scaled_training = Y_scaler.fit_transform(Y_training)

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
#Y_scaled_testing = Y_scaler.transform(Y_testing)

## Core tensorflow 1.5

In [13]:
# Define how many inputs and outputs are in our neural network
number_of_inputs = X_scaled_training.shape[1]
number_of_outputs = 2

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 10
layer_2_nodes = 50
layer_3_nodes = 50
layer_4_nodes = 10

In [9]:
# Section One: Define the layers of the neural network itself

tf.compat.v1.disable_eager_execution()

# Input Layer
with tf.compat.v1.variable_scope('input'):
    X = tf.compat.v1.placeholder(tf.float32, shape=(None, number_of_inputs))

    
# Layer 1
with tf.compat.v1.variable_scope('layer_1'):
    weights = tf.compat.v1.get_variable(
        name='weights1', 
        shape=[number_of_inputs, layer_1_nodes], 
        initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")
    )
    biases = tf.compat.v1.get_variable(
        name='biases1', 
        shape=[layer_1_nodes], 
        initializer=tf.compat.v1.zeros_initializer()
    )
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

    
# Layer 2
with tf.compat.v1.variable_scope('layer_2'):
    weights = tf.compat.v1.get_variable(
        name='weights2', 
        shape=[layer_1_nodes, layer_2_nodes], 
        initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")
    )
    biases = tf.compat.v1.get_variable(
        name='biases2', 
        shape=[layer_2_nodes], 
        initializer=tf.compat.v1.zeros_initializer()
    )
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

    
# Layer 3
with tf.compat.v1.variable_scope('layer_3'):
    weights = tf.compat.v1.get_variable(
        name='weights3', 
        shape=[layer_2_nodes, layer_3_nodes], 
        initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")
    )
    biases = tf.compat.v1.get_variable(
        name='biases3', 
        shape=[layer_3_nodes], 
        initializer=tf.compat.v1.zeros_initializer()
    )
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)
    

# Layer 4
with tf.compat.v1.variable_scope('layer_4'):
    weights = tf.compat.v1.get_variable(
        name='weights4', 
        shape=[layer_3_nodes, layer_4_nodes], 
        initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")
    )
    biases = tf.compat.v1.get_variable(
        name='biases4', 
        shape=[layer_4_nodes], 
        initializer=tf.compat.v1.zeros_initializer()
    )
    layer_4_output = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)
    
    
# Output Layer
with tf.compat.v1.variable_scope('output'):
    weights = tf.compat.v1.get_variable(
        name='weights5', 
        shape=[layer_4_nodes, number_of_outputs], 
        initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform")
    )
    biases = tf.compat.v1.get_variable(
        name='biases5', 
        shape=[number_of_outputs], 
        initializer=tf.compat.v1.zeros_initializer()
    )
    prediction = tf.nn.softmax(tf.matmul(layer_4_output, weights) + biases)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
# Section Two: Define the cost function of the neural network that will measure prediction accuracy during training
with tf.compat.v1.variable_scope('cost'):
    Y = tf.compat.v1.placeholder(tf.float32) # removed shape argument
    #cost = tf.reduce_mean(input_tensor=tf.math.squared_difference(prediction, Y))
    
    cost = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(
        labels=Y,
        logits=prediction,
        pos_weight=class_weight,
        name=None
    ))

In [11]:
# Section Three: Define the optimizer function that will be run to optimize the neural network
learning_rate = 0.001

with tf.compat.v1.variable_scope('train'):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cost)

In [12]:
# Define model parameters
training_epochs = 500

# Initialize a session so that we can run TensorFlow operations
with tf.compat.v1.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.compat.v1.global_variables_initializer())

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_training})

        # Every 50 training steps, log our progress
        if epoch % 50 == 0:
            training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_training})
            testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_testing})
            print(epoch, np.round(training_cost,4), np.round(testing_cost,4))

    # Training is now complete!
    print("Training is complete!")

    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_testing})
    print("Final training/testing cost: {}/{}".format(np.round(training_cost, 4), np.round(testing_cost, 4)))
    
    # Now that the neural network is trained, let's use it to make predictions for our test data.
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted = session.run(prediction, feed_dict={X: X_scaled_training})
    

0 4.9236 4.9242
50 3.8016 3.8041
100 3.7943 3.7968
150 3.7935 3.796
200 3.7932 3.7957
250 3.7931 3.7957
300 3.7931 3.7956
350 3.7931 3.7956
400 3.7931 3.7956
450 3.7931 3.7956
Training is complete!
Final training/testing cost: 3.793100118637085/3.795599937438965


In [ ]:
true = np.argmax(Y_training, axis=1)
pred = np.argmax(Y_predicted, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(true, pred)

sn.set(font_scale=1.4) # for label size
sn.heatmap(cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()